In [6]:
import re
import pickle
import json
import yaml
import constants

In [10]:
CATEGORIES = constants.CATEGORIES

In [11]:
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text


def replace_regex(pattern, replacement, text):
    # Create a regex pattern
    regex = re.compile(pattern)

    # Perform the replacement
    replaced_text = regex.sub(replacement, text)

    return replaced_text


def split_to_cats(text):
    global CATEGORIES
    cats = {}
    for cat in CATEGORIES:
        if ' ' in cat:
            cat_to_pateert = cat.split(' ')
            pattern = rf'\n{cat_to_pateert[0]}\s+{cat_to_pateert[1]}[.](.+?)(?=\n[аА-яЯ]+[.]?[•]?)'
        else:
            pattern = rf'\n{cat}[.](.+?)(?=\n[аА-яЯ]+[.]?[•]?)'
        result = re.findall(pattern, text, re.DOTALL)
        if len(result) > 0:
            result = result[0].strip('.').strip()
            result = result[:10].lower() + result[10:]
            result = result[:1].upper() + result[1:]
        else:
            result = ''
        cats[cat] = result
    return cats


In [12]:
file_path = '../data/dataset/atlas.html'
with open(file_path, 'r') as file:
    doc = file.read()

doc = doc.replace('<br>', '')
pattern = re.compile(
    '<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:.*?px; top:.*?px; width:.*?px; height:.*?px;"><span style="font-family: OFYQTW.+TimesNewRomanPS-BoldMT; font-size:23px">([А-Я\s]+[А-Я]\s*)<\/span>',
    flags=re.M)
splitted_doc = re.split(pattern, doc)

clear_doc = []
for x in splitted_doc:
    ready = remove_html_tags(x)
    ready = ready.replace('-\n', '')
    clear_doc.append(ready)

pattern = r"(?!\nОписание)(?!\nДругие виды)(?!\nДругие  виды)(?!\nАреал)(?!\nЭкология)(?!\nРесурсы)(?!\nВозделывание)(?!\nСырьё)(?!\nХимический состав)(?!\nХимический  состав)(?!\nФармакологические свойства)(?!\nПрименение в медицине)(?!\nЛитература)\n+"

replacement = ""
clear_doc_withoutn = []
for x in clear_doc:
    ready = replace_regex(pattern, replacement, x)\

    clear_doc_withoutn.append(ready)
clear_doc_withoutn = clear_doc_withoutn[3:]


In [13]:
dict_doc = {}
for i in range(0, len(clear_doc_withoutn), 2):
    dict_text = {}
    name = clear_doc_withoutn[i].strip()
    text = clear_doc_withoutn[i + 1]
    description = split_to_cats(text)
    dict_doc[name] = description
with open('../data/dataset/data.yml', 'w', encoding='utf-8') as outfile:
    yaml.dump(dict_doc, outfile, default_flow_style=False, allow_unicode=True)